In [108]:
import pandas as pd
import random as rd

from planning_livreurs import *

from IPython.display import HTML
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [61]:
list_streets = ['Champs-Elysées', 'Convention', 'Sts']

# On crée le dictionnaire avec des VBox vide :
dic_widgets_streets = {key: widgets.VBox() for key in list_streets}
dic_widgets_streets

In [103]:
for key in dic_widgets_streets.keys():
    # On associera le 1er élément de VBox.children comme étant relatif aux vendeurs
    # et le 2nd élément sera lié aux colis.
    dic_widgets_streets[key].children = [
        widgets.HTML(value=f'<b>{key} :</b>'),
        widgets.IntSlider(value=7,
                          min=0,
                          max=10,
                          step=1,
                          description='#vendeurs:'),
        widgets.IntSlider(value=7,
                          min=0,
                          max=10,
                          step=1,
                          description='#colis:')
    ]

hbox_children = list(dic_widgets_streets.values())
hbox = widgets.HBox(hbox_children)
hbox

In [ ]:
def get_list_prenoms_courants(k):
    df_prenoms = pd.read_csv('data/prenoms.csv')
    
    return = rd.sample(list_prenoms, k=k)

In [102]:
update_button = widgets.Button(
    description='Mettre à jour',
)

container = widgets.VBox([hbox ,update_button])

@update_button.on_click
def update(b=None):
    ## Récupération des valeurs des widgets :
    CE_nb_livreurs=dic_widgets_streets['Champs-Elysées'].children[1]
    CE_nb_colis=dic_widgets_streets['Champs-Elysées'].children[2]
    
    Conv_nb_livreurs=dic_widgets_streets['Convention'].children[1]
    Conv_nb_colis=dic_widgets_streets['Convention'].children[2]
    
    Sts_nb_livreurs=dic_widgets_streets['Sts'].children[1]
    Sts_nb_colis=dic_widgets_streets['Sts'].children[2]
    
    # On crée des dictionnaires
    
    
    
    ## Initialisation des variables fixées pour la suite :
    num_weeks = 1
    tagg_heure = ["0:00-1:00", "1:00-2:00", "2:00-3:00", "3:00-4:00","4:00-5:00","5:00-6:00","6:00-7:00","7:00-8:00","8:00-9:00","9:00-10:00","10:00-11:00","11:00-12:00","12:00-13:00","13:00-14:00","14:00-15:00","15:00-16:00","16:00-17:00","17:00-18:00","18:00-19:00","19:00-20:00","20:00-21:00","21:00-22:00","22:00-23:00","23:00-00:00"]
    zones = ["Avenue des Champs Elysées", "Sts-Pères", "Convention"]
    zones_heures={}
    for index, zone in enumerate(zones):
        zones_heures[zone] = [i for i in range(len(tagg_heure)*num_weeks*7)]
    
    
    # On pioche aléatoirement parmi une liste de prénoms :
    livreurs = dict(zip(range(CE_nb_livreurs)), get_list_prenoms_courants(k=CE_nb_livreurs))
    
    
    solve_shift_scheduling(livreurs, Creneaux, num_weeks, zones, max_shift_per_day, 
                           min_shift_per_day, contrainte_tp, daily_cover_demands,occ_rues, 
                           waiting_time,shift_hours)
    
    
    Creneaux = ["R"]+tagg_heure
    max_shift_per_day = 8
    min_shift_per_day= 4
    
    nb_colis_ACE = {0: 3, 1: 1, 2: 2, 3: 1, 4: 1, 5: 4, 6: 2}
    nb_colis_Sts = {0: 2, 1: 2, 2: 2, 3: 4, 4: 1, 5: 1, 6: 2}
    nb_colis_convention = {0: 1, 1: 3, 2: 2, 3: 1, 4: 4, 5: 1, 6: 2}
    
    
    daily_cover_demands={}
    for d in range (num_weeks*7):
        for zone in zones:
            if zone == "Avenue des Champs Elysées":
                daily_cover_demands[d,zone]=nb_colis_ACE[d]
            if zone == "Sts-Pères":
                daily_cover_demands[d,zone]=nb_colis_Sts[d]
            if zone == "Convention":
                daily_cover_demands[d,zone]=nb_colis_convention[d]
                
    # paramètres temps de travails
    # contrainte_tp = [ [task1, task2,....], [shift1, shift2,...], total_nb_of_hours, nb_of_hours_min, nb_of_hours_max, min_penalty, max_penalty]
    contrainte_tp = []
    
    contrainte_tp.append([zone for zone in zones])
    contrainte_tp.append([shifts for shifts in range(len(tagg_heure))])
    # base_pourc_work
    contrainte_tp.append(35)
    contrainte_tp.append(0)
    contrainte_tp.append(60)
    contrainte_tp.append(15)
    contrainte_tp.append(15)

    shift_hours = {"0:00-1:00" : 1, "1:00-2:00":1, "2:00-3:00":1, "3:00-4:00":1,
                   "4:00-5:00":1,"5:00-6:00":1,"6:00-7:00":1,"7:00-8:00":1,
                   "8:00-9:00":1,"9:00-10:00":1,"10:00-11:00":1,"11:00-12:00":1,
                   "12:00-13:00":1,"13:00-14:00":1,"14:00-15:00":1,"15:00-16:00":1,
                   "16:00-17:00":1,"17:00-18:00":1,"18:00-19:00":1,"19:00-20:00":1,
                   "20:00-21:00":1,"21:00-22:00":1,"22:00-23:00":1,"23:00-00:00":1}

    df_ACE = pd.read_csv(os.path.join(r"comptages-routiers-permanents_ACE.csv"), sep=";")
    df_Sts = pd.read_csv(os.path.join(r"comptages-routiers-permanents_Sts.csv"), sep=";")
    df_convention = pd.read_csv(os.path.join(r"comptages-routiers-permanents_convention.csv"), sep=";")
    
    debut = pd.Timestamp('2020-11-23')
    fin = pd.Timestamp('2020-11-30')
    
    
    
    for df in (df_ACE, df_Sts, df_convention):
        df['Date et heure de comptage'] = df['Date et heure de comptage'].apply(lambda s: clean_date(s))
    for df in (df_ACE, df_Sts, df_convention):
        df['Date et heure de comptage'] = pd.to_datetime(df["Date et heure de comptage"], format='%Y-%m-%d %H:%M:%S')
    for df in (df_ACE, df_Sts, df_convention):
        df.sort_values("Date et heure de comptage", inplace=True)


    df_ACE = df_ACE[(df_ACE['Date et heure de comptage'] >= debut)]
    df_ACE = df_ACE[(df_ACE['Date et heure de comptage'] <= fin)]
    tx_occup_ACE = np.array(df_ACE["Taux d'occupation"])
    debit_ACE = np.array(df_ACE['Débit horaire'])

    print(df_ACE['Date et heure de comptage'])
    df_Sts = df_Sts[(df_Sts['Date et heure de comptage'] >= debut)]
    df_Sts = df_Sts[(df_Sts['Date et heure de comptage'] <= fin)]
    tx_occup_Sts = np.array(df_Sts["Taux d'occupation"])
    debit_Sts = np.array(df_Sts['Débit horaire'])

    df_convention = df_convention[(df_convention['Date et heure de comptage'] >= debut)]
    df_convention = df_convention[(df_convention['Date et heure de comptage'] <= fin)]
    tx_occup_convention = np.array(df_convention["Taux d'occupation"])
    debit_convention = np.array(df_convention['Débit horaire'])
    print(len(tx_occup_convention))

    occ_rues = {}
    for zone in zones:
        for d in range (num_weeks*7):
            for s in range(len(tagg_heure)):
                if zone == "Avenue des Champs Elysées":
                    occ_rues[s, d, zone] = int((tx_occup_ACE[d*len(tagg_heure)+s]/debit_ACE[d*len(tagg_heure)+s])*1000)
                if zone == "Sts-Pères":
                    occ_rues[s, d, zone] = int((tx_occup_Sts[d*len(tagg_heure)+s]/debit_Sts[d*len(tagg_heure)+s])*1000)
                if zone == "Convention":
                    occ_rues[s, d, zone] = int((tx_occup_convention[d * len(tagg_heure) + s] / debit_convention[d * len(tagg_heure) + s])*1000)

    waiting_time = 60
    
    
    
    ## Appel de la fonction de résolution du schedule
    solver, work, status = solve_shift_scheduling(livreurs, Creneaux, num_weeks, zones, 
                                                  max_shift_per_day, min_shift_per_day, 
                                                  contrainte_tp, daily_cover_demands,
                                                  occ_rues, waiting_time,shift_hours)

    
    ## Export du fichier Excel résultant
    all_shifts=num_weeks * len(tagg_heure) * 7
    wb = print_solution_blockwise_del(zones, all_shifts, livreurs, solver, work, tagg_heure)
    wb.save('planning.xls')
    
    
    ## Lecture et affichage du Excel obtenu :
    df = pd.read_excel('planning_test.xls', skiprows=3)
    
    df = df.fillna('')
    df.to_html('planning.html')
    
    display(HTML(filename="planning.html"))
    
    return

display(container)

,Unnamed: 0,Unnamed: 1,Avenue des Champs Elysées,Sts-Pères,Convention
0,,0:00-1:00,,,
1,,1:00-2:00,,,
2,,2:00-3:00,,,Matthias
3,,3:00-4:00,,,
4,,4:00-5:00,,,
5,,5:00-6:00,Carlos,,
6,,6:00-7:00,,,
7,,7:00-8:00,,,
8,,8:00-9:00,livreur_3,,
9,,9:00-10:00,Carlos,,
